In [2]:
pip install fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [102]:
import requests # api 호출 모듈
from fake_useragent import UserAgent # 403에러 방지 모듈
import time # 시간 모듈
import random # 랜덤 모듈
import concurrent.futures # 병렬처리 모듈
import os
import pandas as pd
import json
# 사용자 에이전트 설정
ua = UserAgent()
headers = {
    "User-Agent": ua.random
}


In [19]:
df=pd.read_csv("./sec_edgar_bank_section_10_k.csv")
df.isna().sum()

Company Name       0
Symbol            18
CIK                0
Filling Date       0
Link            1578
dtype: int64

In [27]:
df2 = df.loc[df.Link.isna()==False]

In [29]:
df2

,Company Name,Symbol,CIK,Filling Date,Link
0,AMERICAN EXPRESS CO,AXP,4962,2024-02-09,/cgi-bin/viewer?action=view&cik=4962&accession...
1,AMERICAN EXPRESS CO,AXP,4962,2023-02-10,/cgi-bin/viewer?action=view&cik=4962&accession...
2,AMERICAN EXPRESS CO,AXP,4962,2022-02-11,/cgi-bin/viewer?action=view&cik=4962&accession...
3,AMERICAN EXPRESS CO,AXP,4962,2021-02-12,/cgi-bin/viewer?action=view&cik=4962&accession...
4,AMERICAN EXPRESS CO,AXP,4962,2020-02-13,/cgi-bin/viewer?action=view&cik=4962&accession...
...,...,...,...,...,...
2920,Cadence Bancorporation,CADE,1614184,2021-03-01,/cgi-bin/viewer?action=view&cik=1614184&access...
2921,Cadence Bancorporation,CADE,1614184,2020-02-28,/cgi-bin/viewer?action=view&cik=1614184&access...
2923,Cadence Bancorporation,CADE,1614184,2019-03-08,/cgi-bin/viewer?action=view&cik=1614184&access...
2924,Cadence Bancorporation,CADE,1614184,2019-03-01,/cgi-bin/viewer?action=view&cik=1614184&access...


In [340]:
# 파일 다운로드 함수
def heavy_work(Ticker,CIK,FD,Url):
    
    url=Url.split('accession_number=')[1].split("&xbrl_type=v")[0]
    url=url.split("-")
    sum_url = url[0]+url[1]+url[2]
    fixed_cik = str(CIK).zfill(10)
    fixed_url = f"https://www.sec.gov/Archives/edgar/data/{fixed_cik}/{sum_url}/Financial_Report.xls"
    # 딜레이 추가
    
    # # url state 확인
    # # 사용자 에이전트 설정
    ua = UserAgent()
    headers = {
            "Host": "www.sec.gov",
            "Sec-Fetch-Site": "none",
            "Accept-Language": "ko-KR,ko;q=0.9",
            "Connection": "keep-alive",
            "Sec-Fetch-Mode": "navigate",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
            "Sec-Fetch-Dest": "document",
            "Accept-Encoding": "gzip, deflate, br",
        }

    response = requests.get(fixed_url, headers=headers)
    delay = random.random()+1
    time.sleep(delay)
    # # state가 200일 경우
    try:
        fixed_ticker = Ticker.split("/")[0]
    except:
        fixed_ticker = Ticker
    directory=f"{fixed_ticker}_{CIK}"
    if response.status_code == 200:
        #디랙토리 생성
        if not os.path.exists(directory):
            os.makedirs(directory)
        #다음 출력의 파일이 저장됨
        with open(f"./{directory}/{fixed_ticker}_{CIK}_{FD}_10_K.xls", "wb") as file:
            file.write(response.content)
            
        # 완료 표기
        print(f"File downloaded successfully, {fixed_ticker}_{CIK}_{FD}_10_K.xls")
        return;
    else:
        print(fixed_url, response.status_code)
        return fixed_url

In [247]:
error_list=[]
for i in df2.values[1200:]:
    error_url = heavy_work(i[1],i[2],i[3],i[4])
    error_list.append(error_url)

File downloaded successfully, AX_1299709_2018-08-23_10_K.xlsx
File downloaded successfully, AX_1299709_2017-08-24_10_K.xlsx
File downloaded successfully, AX_1299709_2016-08-25_10_K.xlsx
File downloaded successfully, AX_1299709_2015-08-26_10_K.xlsx
File downloaded successfully, AX_1299709_2014-08-28_10_K.xlsx
https://www.sec.gov/Archives/edgar/data/0001299709/000129970913000135/Financial_Report.xlsx 404
https://www.sec.gov/Archives/edgar/data/0001299709/000129970912000126/Financial_Report.xlsx 404
File downloaded successfully, HOMB_1331520_2024-02-26_10_K.xlsx
File downloaded successfully, HOMB_1331520_2023-02-24_10_K.xlsx
File downloaded successfully, HOMB_1331520_2022-02-24_10_K.xlsx
File downloaded successfully, HOMB_1331520_2021-02-26_10_K.xlsx
File downloaded successfully, HOMB_1331520_2020-02-26_10_K.xlsx
File downloaded successfully, HOMB_1331520_2019-02-26_10_K.xlsx
File downloaded successfully, HOMB_1331520_2018-02-27_10_K.xlsx
File downloaded successfully, HOMB_1331520_2017-02

In [272]:
total_list = os.listdir("./")
CIK_list = []
for i in total_list:
    if '.' not in i:
        CIK_list.append(i)

In [284]:
file_count =[]
for i in CIK_list:
    count = len(os.listdir(f"./{i}"))
    if count < 6:
        print(i)
    file_count.append(count)

CADE_1614184


In [336]:
# df3=df2["Filling Date"].str.slice(0,4)
# df3 = .astype("int32")
# df3
df4 = df2.loc[df2["Filling Date"].str.slice(0,4) < "2014"]
# df5 = df2.loc[df4]
df4

,Company Name,Symbol,CIK,Filling Date,Link
11,AMERICAN EXPRESS CO,AXP,4962,2013-02-22,/cgi-bin/viewer?action=view&cik=4962&accession...
13,AMERICAN EXPRESS CO,AXP,4962,2012-02-24,/cgi-bin/viewer?action=view&cik=4962&accession...
14,AMERICAN EXPRESS CO,AXP,4962,2011-02-28,/cgi-bin/viewer?action=view&cik=4962&accession...
15,AMERICAN EXPRESS CO,AXP,4962,2010-02-26,/cgi-bin/viewer?action=view&cik=4962&accession...
46,ASSOCIATED BANC-CORP,ASB,7789,2013-02-15,/cgi-bin/viewer?action=view&cik=7789&accession...
...,...,...,...,...,...
2830,"MidWestOne Financial Group, Inc.",MOFG,1412665,2012-03-09,/cgi-bin/viewer?action=view&cik=1412665&access...
2864,"Customers Bancorp, Inc.",CUBI,1488813,2013-03-18,/cgi-bin/viewer?action=view&cik=1488813&access...
2865,"Customers Bancorp, Inc.",CUBI,1488813,2012-03-21,/cgi-bin/viewer?action=view&cik=1488813&access...
2877,"BankUnited, Inc.",BKU,1504008,2013-02-25,/cgi-bin/viewer?action=view&cik=1504008&access...


In [342]:
for i in df4.values:
    error_url = heavy_work(i[1],i[2],i[3],i[4])
    error_list.append(error_url)

File downloaded successfully, AXP_4962_2013-02-22_10_K.xls
File downloaded successfully, AXP_4962_2012-02-24_10_K.xls
File downloaded successfully, AXP_4962_2011-02-28_10_K.xls
File downloaded successfully, AXP_4962_2010-02-26_10_K.xls
File downloaded successfully, ASB_7789_2013-02-15_10_K.xls
File downloaded successfully, ASB_7789_2012-02-28_10_K.xls
File downloaded successfully, ASB_7789_2011-02-15_10_K.xls
File downloaded successfully, SNV_18349_2013-03-01_10_K.xls
File downloaded successfully, SNV_18349_2012-02-29_10_K.xls
File downloaded successfully, SNV_18349_2011-03-01_10_K.xls
File downloaded successfully, JPM_19617_2013-02-28_10_K.xls
File downloaded successfully, JPM_19617_2012-02-29_10_K.xls
File downloaded successfully, JPM_19617_2011-02-28_10_K.xls
File downloaded successfully, JPM_19617_2010-02-24_10_K.xls
File downloaded successfully, CBSH_22356_2013-02-22_10_K.xls
File downloaded successfully, CBSH_22356_2012-02-22_10_K.xls
File downloaded successfully, CBSH_22356_2011